In [4]:
import pandas as pd
import os
import requests
import re
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
from pandas_datareader import data
import pandas_datareader as web
from bs4 import BeautifulSoup
import yaml
import json


# Yahoo Finance WEBSCRAPER

In [ ]:
stocks = ['GOOGL','AAPL','TSLA','AMZN','META', 'SNAP']
User_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'

In [ ]:
# Current Stock prices
base_url = 'https://finance.yahoo.com/quote/'
tickr_url = [f'{x}?p={x}&.tsrc=fin-srch' for x in stocks]

current_price = []
missing_data = []
print('Getting current prices for:')
for tickr,stock in zip(tickr_url,stocks):
    print(f'...{stock}...')
    try:
        content = requests.get(base_url+tickr,headers={'User-Agent':User_agent}).content
        soup = BeautifulSoup(content)
        soup = soup.find('fin-streamer',class_='Fw(b) Fz(36px) Mb(-4px) D(ib)') #Fw(b) Fz(36px) Mb(-4px) D(ib)
        current_price.append(float(soup['value']))
    except:
        print(f'No data for {stock}')
        missing_data.append(stock)
        current_price.append('Na')

pd.DataFrame({'Ticker':stocks, 'Current Price':current_price})

In [ ]:
# 20Y Historical Stock Prices
base_url = 'https://query1.finance.yahoo.com/v7/finance/download/{}?' # 'AAPL?period1=1645596185&amp;period2=1677132185&amp;interval=1d&amp;events=history&amp;includeAdjustedClose=true'
params = {'range': '20y',
          'interval': '1d',
          'events':'history'}
print('Getting ({}) historical prices for:'.format(params['range']))
for stock in stocks:
    print(f'...{stock}...')
    try:
        os.mkdir('./Historical/{}/{}'.format(params['range'],stock))
        print('New directory completed')
    except FileExistsError:
        print('A directory already exists, additional data for {} is being saved'.format(stock))
    try:
        response = requests.get(base_url.format(stock), params=params,headers={'User-Agent': User_agent}).content
        soup = BeautifulSoup(response)
        ls = str(soup.find('p')).replace('<p>','').splitlines()
        cols = ls[0]
        data = np.array([x.split(',') for x in ls[1:]])
        pd.DataFrame(data=data,columns=cols.split(',')).to_csv('./Historical/{}/{}/historical_price.csv'.format(params['range'],stock))
    except:
        print(f'No data for {stock}')


# SEC DATA


In [ ]:
SEC_user_agent = 'matiasrhuber@gmail.com'
encoding = 'gzip, deflate'
host = 'www.sec.gov'
SEC_headers={'User-Agent': SEC_user_agent,'Accept-Encoding':'gzip','Host':host}
CIK = '320193'
years = range(2000,2023)
params = {'action':'getcompany',
          'CIK': '789019',
          'type': '10-k', #optional
          'dateb': '20190101', #optional
          #'datea': 20220101, #optional
          'owner': 'exclude', # default set to exclude
          'start':'',
          'output': 'atom',
          'count': '100' #number of results I want to see default is 40
          } #### Never worked :(

In [ ]:
# Ticker and CIK mapping
tickers_cik = requests.get("https://www.sec.gov/files/company_tickers.json",headers=SEC_headers)
tickr_text = tickers_cik.text
tickr_dict = json.loads(tickr_text)

for num in range(len(tickr_dict)):
    tickr_dict[(tickr_dict[str(num)]["ticker"])] = tickr_dict[str(num)]
    del tickr_dict[str(num)]
# for tickr in tickr_dict:
#     if tickr_dict[tickr]['cik_str'] == '1265107':
#         print(tickr)
# POTENTIAL MISSING DATA

In [ ]:
# Index for file locations of 10-k and 10-q filings
base_url = 'https://www.sec.gov/Archives/edgar/full-index/'
quarters = ['QTR1','QTR2','QTR3','QTR4']
years = range(2000,2022) 
for year in years:
    try:
        os.mkdir(f'./SEC/Master_Index/{year}')
    except:
        print(f'Previous data from year {year} already saved')
    for q in quarters:
        print(f'Going through files from {year} {q}...')
        master_index = requests.get(base_url+f'{year}/{q}/master.idx', headers=SEC_headers).text
        master_index = master_index.split('--------------------------------------------------------------------------------')[1]
        master_index = master_index.replace('\n','|').split('|')
        del master_index[0]
        keys = master_index[::5]
        del master_index[::5]

        # Fit data into dictionary and select only 10-k and 10-q file types
        shape = int(len(master_index)/4)
        print(f'File size for {year} {q}: {shape}')
        
        data = np.array(master_index).reshape(shape,4)
        cols = ['Company Name', 'Form Type', 'Date Filled', 'File Name']
        dict_master = {}
        for set,key in zip(data,keys):
            dict_temp = {key:value for (key,value) in zip(cols,set)}
            if dict_temp['Form Type'] == '10-K':
                dict_master[key+'_10-k'] = dict_temp
            if dict_temp['Form Type'] == '10-Q':
                dict_master[key+'_10-q'] = dict_temp
        print(f'10-K or 10-Q Filings saved: {len(dict_master)}')
        
        with open(f'./SEC/Master_Index/{year}/{q}.txt','w') as f:
            f.write(json.dumps(dict_master))
   

In [ ]:
base_url = 'https://www.sec.gov/Archives/'
# stock = stocks[0]
file_types = ['_10-q','_10-k']
quarters = ['QTR1','QTR2','QTR3','QTR4']
years = ['2020','2021','2022']
curr_dir = os.getcwd()
folder = 'SEC'
# ocf = ['Net cash provided by operating activities', 'Cash generated by operating activities']
# title = 'CONSOLIDATED STATEMENTS OF CASH FLOWS'
# oex = ['Purchases of property and equipment', 'Payments for acquisition of property, plant and equipment']
# for year in years:
#     for quarter in quarters:
year = '2020'
quarter = 'QTR1'
file_type = '_10-q'
with open(f'./SEC/Master_Index/{year}/{quarter}.txt','r') as f: # Apply different years and quarters
    text = f.read()
dict_ind = json.loads(text)
stocks = 'GOOGL'
for stock in stocks:
    cik = tickr_dict[stock]['cik_str']
    try:
        file_type = file_types[0] # Quarterly Filings
        url = dict_ind[str(cik)+file_type]['File Name'].replace('-','').replace('.txt','')
    except:
        file_type = file_types[1] # Yearly Filings
        url = dict_ind[str(cik)+file_type]['File Name'].replace('-','').replace('.txt','')
    file = '/FilingSummary.xml'
    data = requests.get(base_url+url+file,headers=SEC_headers).content
    soup = BeautifulSoup(data, 'lxml')
    myreports = soup.find('myreports')

    # List with individual components from myreports
    master_reports = []
    print(f'Gathering data for {stock}{file_type} @ {quarter} {year}')
    for report in myreports.find_all('report')[:-1]:

    # dictionary with all relevant parts
        report_dict = {}
        report_dict['name_short'] = report.shortname.text
        report_dict['name_long'] = report.longname.text
        report_dict['position'] = report.position.text
        report_dict['category'] = report.menucategory.text
        report_dict['url'] = base_url + url + '/' + report.htmlfilename.text
        master_reports.append(report_dict)
        print(report_dict['name_short'])

    try: 
        os.mkdir(os.path.join(curr_dir,folder,stock))
    except:
        print('Saving master report along existing data {}'.format(stock))
    with open(os.path.join(curr_dir,folder,stock,f'master_reports_{file_type}_{year}_{quarter}.json'), 'w') as fout:
        json.dump(master_reports , fout)

In [ ]:
financial_statements_mapping = {
    "Consolidated Balance Sheets": [
        "Consolidated Balance Sheets",
        "CONSOLIDATED BALANCE SHEETS",
        "Condensed Consolidated Balance Sheets",
        "CONDENSED CONSOLIDATED BALANCE SHEETS",
        "Consolidated Balance Sheets (Unaudited)",
        "CONSOLIDATED BALANCE SHEETS (Unaudited)",
        "Condensed Consolidated Balance Sheets (Unaudited)",
        "CONDENSED CONSOLIDATED BALANCE SHEETS (Unaudited)",
        "Balance Sheet",
        "BALANCE SHEET",
        "Statement of Financial Position",
        "STATEMENT OF FINANCIAL POSITION",
        "Consolidated Statements of Financial Position",
        "CONSOLIDATED STATEMENTS OF FINANCIAL POSITION",
        "Consolidated Statements of Financial Condition",
        "CONSOLIDATED STATEMENTS OF FINANCIAL CONDITION",
        "Consolidated Statements of Position",
        "CONSOLIDATED STATEMENTS OF POSITION",
        "Financial Position",
        "FINANCIAL POSITION"
    ],
    "Consolidated Statements of Operations": [
        "Consolidated Statements of Operations",
        "CONSOLIDATED STATEMENTS OF OPERATIONS",
        "Consolidated Statements Of Operations",
        "Consolidated Statements of Operations (Unaudited)",
        "CONSOLIDATED STATEMENTS OF OPERATIONS (Unaudited)",
        "Condensed Consolidated Statements of Operations (Unaudited)",
        "CONDENSED CONSOLIDATED STATEMENTS OF OPERATIONS (Unaudited)",
        "Consolidated Statements of Income",
        "CONSOLIDATED STATEMENTS OF INCOME",
        "Condensed Consolidated Statements of Income",
        "CONDENSED CONSOLIDATED STATEMENTS OF INCOME",
        "Income Statement",
        "INCOME STATEMENT",
        "Statement of Earnings",
        "STATEMENT OF EARNINGS",
        "Profit and Loss Statement (P&L)",
        "PROFIT AND LOSS STATEMENT (P&L)",
        "Statement of Comprehensive Income",
        "STATEMENT OF COMPREHENSIVE INCOME",
        "Statement of Operations",
        "STATEMENT OF OPERATIONS",
        "Statement of Income",
        "STATEMENT OF INCOME",
        "Earnings Statement",
        "EARNINGS STATEMENT",
        "Revenue and Expense Statement",
        "REVENUE AND EXPENSE STATEMENT",
        "Operating Statement",
        "OPERATING STATEMENT",
        "Statement of Profit and Loss and Other Comprehensive Income",
        "STATEMENT OF PROFIT AND LOSS AND OTHER COMPREHENSIVE INCOME",
        "Statement of Earnings and Retained Earnings",
        "STATEMENT OF EARNINGS AND RETAINED EARNINGS",
        "Statement of Income and Expenditure",
        "STATEMENT OF INCOME AND EXPENDITURE"
    ],
    "Consolidated Statements of Cash Flows": [
        "Consolidated Statements of Cash Flows",
        "CONSOLIDATED STATEMENTS OF CASH FLOWS",
        "Consolidated Statements of Cash Flows (Unaudited)",
        "CONDENSED CONSOLIDATED STATEMENTS OF CASH FLOWS (Unaudited)",
        "Condensed Consolidated Statements of Cash Flows",
        "CONDENSED CONSOLIDATED STATEMENTS OF CASH FLOWS",
        "Cash Flow Statement",
        "CASH FLOW STATEMENT",
        "Statement of Cash Flows",
        "STATEMENT OF CASH FLOWS",
        "Statement of Changes in Cash",
        "STATEMENT OF CHANGES IN CASH",
        "Statement of Cash Flow",
        "STATEMENT OF CASH FLOW"
    ],
    "Consolidated Statements of Shareholders' Equity": [
        "Consolidated Statements of Shareholders' Equity",
        "CONSOLIDATED STATEMENTS OF SHAREHOLDERS' EQUITY",
        "Condensed Consolidated Statements of Shareholders' Equity (Unaudited)",
        "CONDENSED CONSOLIDATED STATEMENTS OF SHAREHOLDERS' EQUITY (Unaudited)",
        "Consolidated Statements of Redeemable Noncontrolling Interest and Stockholders' Equity (Unaudited)",
        "CONSOLIDATED STATEMENTS OF REDEEMABLE NONCONTROLLING INTEREST AND STOCKHOLDERS' EQUITY (Unaudited)",
        "Consolidated Statements of Redeemable Noncontrolling Interest and Stockholders' Equity",
        "CONSOLIDATED STATEMENTS OF REDEEMABLE NONCONTROLLING INTEREST AND STOCKHOLDERS' EQUITY",
        "Consolidated Statements of Stockholders' Equity",
        "CONSOLIDATED STATEMENTS OF STOCKHOLDERS' EQUITY",
        "Condensed Statements of Shareholders' Equity",
        "CONDENSED STATEMENTS OF SHAREHOLDERS' EQUITY",
        "Statement of Shareholders' Equity",
        "STATEMENT OF SHAREHOLDERS' EQUITY",
        "Shareholders' Equity Statement",
        "STOCKHOLDERS' EQUITY",
        "Stockholders' Equity",
        "SHAREHOLDERS' EQUITY STATEMENT",
        "Equity Statement",
        "EQUITY STATEMENT"
        
    ]
}


In [ ]:
# Earnings files are collected with desired data
### Figure out all possible formatting for the document names ###
stocks = ['GOOGL','AAPL','TSLA','AMZN','META', 'SNAP']
file_types = ['_10-q','_10-k']
quarters = ['QTR1','QTR2','QTR3','QTR4']
years = ['2020','2021','2022']
curr_dir = os.getcwd()
folder = 'SEC'
statements_url = []
# item1 = [r"Consolidated Balance Sheets", r"CONSOLIDATED BALANCE SHEETS", r"CONDENSED CONSOLIDATED BALANCE SHEETS (Unaudited)"]   #"CONSOLIDATED BALANCE SHEETS"
# item2 = [r"Consolidated Statements of Operations", r"CONDENSED CONSOLIDATED STATEMENTS OF OPERATIONS (Unaudited)", r"CONSOLIDATED STATEMENTS OF INCOME", r"Consolidated Statements Of Operations (Unaudited)", r"CONSOLIDATED STATEMENTS OF OPERATIONS"]   #"CONDENSED CONSOLIDATED STATEMENTS OF OPERATIONS (Unaudited)"
# item3 = [r"Consolidated Statements of Cash Flows (Unaudited)", r"CONSOLIDATED STATEMENTS OF CASH FLOWS", r"CONDENSED CONSOLIDATED STATEMENTS OF CASH FLOWS (Unaudited)", r"Consolidated Statements of Cash Flows"] #"CONSOLIDATED STATEMENTS OF CASH FLOWS" #CONDENSED CONSOLIDATED STATEMENTS OF CASH FLOWS (Unaudited)"
# item4 = [r"Consolidated Statements of Redeemable Noncontrolling Interest and Stockholders' Equity", r"CONDENSED CONSOLIDATED STATEMENTS OF SHAREHOLDERS' EQUITY (Unaudited)", r"CONSOLIDATED STATEMENTS OF STOCKHOLDERS' EQUITY", r"Consolidated Statements of Stockholders' Equity", r"CONSOLIDATED STATEMENTS OF SHAREHOLDERS' EQUITY"]   #"CONDENSED CONSOLIDATED STATEMENTS OF SHAREHOLDERS' EQUITY (Unaudited)"
   
missing_data_ls = [] 
num_missing = 0

for stock in stocks:
    try:
        os.mkdir(os.path.join(curr_dir,folder,stock,'financial_statements_url'))
    except:
        print('...previous financial statements already exist...')
    for year in years:
        for quarter in quarters:
            print('*'*100)
            print(f'{stock} {year} {quarter}')
            try:
                file_type = file_types[0]
                with open(os.path.join(curr_dir,folder,stock,'master_reports',f'master_reports_{file_type}_{year}_{quarter}.json')) as f:
                    master_reports = json.load(f)
            except:
                file_type = file_types[1]
                with open(os.path.join(curr_dir,folder,stock,'master_reports',f'master_reports_{file_type}_{year}_{quarter}.json')) as f:
                    master_reports = json.load(f)

            missing_data = [item for item in financial_statements_mapping]
            # THIS NEEDS TO BE DONE WITH RE ### this text may change slightly per quarter
            statements_url = []
            dict_url = {}
            for report_dict in master_reports:
                
                report_list = financial_statements_mapping 
                for item in report_list:
                    if report_dict['name_short'] in financial_statements_mapping[item]:                        

                        print('-'*100)
                        print(report_dict['name_short'])
                        print(report_dict['url'])
                        
                        
                        # print(item)
                        # print(report_dict['name_short'])
                        # print(missing_data)
                        try:
                            missing_data.remove(item)
                            statements_url.append(report_dict['url'])
                            dict_url[item] = report_dict['url']
                        except ValueError:
                            print(f'MULTIPLE REPORTS FOUND FOR: {item}')
            
            with open(os.path.join(curr_dir,folder,stock,'financial_statements_url',f'url{file_type}_{year}_{quarter}.json'), 'w') as f:
                json.dump(dict_url,f)
                
                        

                    # elif report_dict['name_short'] in ['CONDENSED ' + x for x in financial_statements_mapping[item]]:
                    
                    #     print('-'*100)
                    #     print(report_dict['name_short'])
                    #     print(report_dict['url'])
                        
                    #     statements_url.append(report_dict['url'])
                    #     missing_data.remove(item)
            if len(missing_data) != 0:
                num_missing += len(missing_data)
                missing_data_ls.append(f'Missing Data for {stock}, {year}, {quarter}, {file_type} \n {len(missing_data)}')
                
                
for x in missing_data_ls:
    print(x)
    
print(f'Total: {num_missing}')
print(statements_url)

In [ ]:
with open(f'./SEC/AMZN/financial_statements_url/url_10-q_2020_QTR2.json','r') as f: 
    text = f.read()
    statements_url = json.loads(text)
print(statements_url)

with open(f'./SEC/GOOGL/financial_statements_url/url_10-q_2020_QTR2.json','r') as f: 
    text = f.read()
    statements_url = json.loads(text)
print(statements_url)

with open(f'./SEC/TSLA/financial_statements_url/url_10-q_2020_QTR2.json','r') as f: 
    text = f.read()
    statements_url = json.loads(text)
print(statements_url)

In [ ]:
# let's assume we want all the statements in a single data set.
statements_data = []

# loop through each statement url
for statement in statements_url.values():

    # define a dictionary that will store the different parts of the statement.
    statement_data = {}
    statement_data['headers'] = []
    statement_data['sections'] = []
    statement_data['data'] = []
    
    # request the statement file content
    content = requests.get(statement, headers=SEC_headers).content
    report_soup = BeautifulSoup(content, 'lxml')

    # find all the rows, figure out what type of row it is, parse the elements, and store in the statement file list.
    for index, row in enumerate(report_soup.table.find_all('tr')):
        
        # first let's get all the elements.
        cols = row.find_all('td')
        
        # if it's a regular row and not a section or a table header
        if (len(row.find_all('th')) == 0 and len(row.find_all('strong')) == 0): 
            reg_row = [ele.text.strip() for ele in cols]
            statement_data['data'].append(reg_row)
            
        # if it's a regular row and a section but not a table header
        elif (len(row.find_all('th')) == 0 and len(row.find_all('strong')) != 0):
            sec_row = cols[0].text.strip()
            statement_data['sections'].append(sec_row)
            
        # finally if it's not any of those it must be a header
        elif (len(row.find_all('th')) != 0):            
            hed_row = [ele.text.strip() for ele in row.find_all('th')]
            statement_data['headers'].append(hed_row)
            
        else:            
            print('We encountered an error.')

    # append it to the master list.
    statements_data.append(statement_data)

In [ ]:
statements_data[2]['headers'][0]

In [ ]:
l = statements_data[0]['headers'][0]
[ item for item in l for _ in range(2) ]
n = 2
file_type = '_10-q'
period_mapping = {'_10-q' : '3 Months Ended', '_10-k' : '12 Months Ended'}

if len(statements_data[n]['headers']) != 1:
    subheaders = statements_data[n]['headers'][1]
    overheaders = statements_data[n]['headers'][0][1:]
    mult = int(len(subheaders)/len(overheaders))
    overheaders_table = [item for item in overheaders for _ in range(mult)]
    income_header = [x+' ('+y+')' for x,y in zip(subheaders,overheaders_table)]
else:
    subheaders = statements_data[n]['headers'][0][1:]
    income_header = [x+' ('+period_mapping[file_type]+')' for x in subheaders]

print(income_header)

In [ ]:
curr_dir = os.getcwd()
folder = 'SEC'
data_dir = os.path.join(curr_dir,folder,'AAPL','financial_statements_raw')
for data in os.listdir(data_dir):
    with open(os.path.join(data_dir,data)) as f:
        statements_data = json.load(f)

In [ ]:
statements_data[2]['headers'][0][0]

In [ ]:
statements_data[0]['headers'][0][0]

In [ ]:
# Grab the proper components
i = 3
### Figure out all possible formatting according to document title, number of headers, and display table accordingly ###

if len(statements_data[i]['headers']) != 1:
    subheaders = statements_data[i]['headers'][1]
    overheaders = statements_data[i]['headers'][0][1:]
    mult = int(len(subheaders)/len(overheaders))
    overheaders_table = [item for item in overheaders for _ in range(mult)]
    income_header = [x+' ('+y+')' for x,y in zip(subheaders,overheaders_table)]
else:
    subheaders = statements_data[i]['headers'][0][1:]
    income_header = [x+' ('+period_mapping[file_type]+')' for x in subheaders]

# income_header =  statements_data[i]['headers'][-1] #threee months ended gives different formatting #[0][1:]
income_data = statements_data[i]['data']

# Put the data in a DataFrame
income_df = pd.DataFrame(income_data)

# Display
print('-'*100)
print('Before Reindexing')
print('-'*100)
display(income_df.head())

# Define the Index column, rename it, and we need to make sure to drop the old column once we reindex.
income_df.index = income_df[0]
income_df.index.name = 'Category'
income_df = income_df.drop(0, axis = 1)

# Display
print('-'*100)
print('Before Regex')
print('-'*100)
display(income_df.head())

# Get rid of the '$', '(', ')', and convert the '' to NaNs.
income_df = income_df.replace('[\$,)]','', regex=True )\
                     .replace( '[(]','-', regex=True)\
                     .replace( '', 'NaN', regex=True)

# Display
print('-'*100)
print('Before type conversion')
print('-'*100)
display(income_df.head())

# everything is a string, so let's convert all the data to a float.
df_copy = income_df.copy()
income_df = income_df.astype(float)

# Change the column headers
print(income_header)
income_df.columns = income_header

# Display
print('-'*100)
print('Final Product')
print('-'*100)

# show the df
display(income_df)

current_dir = os.getcwd()
try:
    folder_dir = os.path.join(current_dir,'SEC',stock)
    os.mkdir(folder_dir)
except:
    print(f'Saving over data for {stock} in {year} {quarter}...')
income_df.to_csv(os.path.join(folder_dir,f"{year+quarter+file_type}.csv"))


In [ ]:
df_copy

In [ ]:
# Grab the proper components

### Figure out all possible formatting according to document title, number of headers, and display table accordingly ###
income_header =  statements_data[1]['headers'][1] #threee months ended gives different formatting #[0][1:]
income_data = statements_data[1]['data']

# Put the data in a DataFrame
income_df = pd.DataFrame(income_data)

# Define the Index column, rename it, and we need to make sure to drop the old column once we reindex.
income_df.index = income_df[0]
income_df.index.name = 'Category'
income_df = income_df.drop(0, axis = 1)

# Get rid of the '$', '(', ')', and convert the '' to NaNs.
income_df = income_df.replace('[\$,)]','', regex=True )\
                     .replace( '[(]','-', regex=True)\
                     .replace( '', 'NaN', regex=True)

# everything is a string, so let's convert all the data to a float.
income_df = income_df.astype(float)

# Change the column headers
income_df.columns = income_header

# show the df
display(income_df)

current_dir = os.getcwd()
try:
    folder_dir = os.path.join(current_dir,'SEC',stock)
    os.mkdir(folder_dir)
except:
    print(f'Saving over data for {stock} in {year} {quarter}...')
income_df.to_csv(os.path.join(folder_dir,f"{year+quarter+file_type}.csv"))


In [ ]:
statements_data[3]

In [ ]:
income_df.to_csv

In [ ]:
# Search for Filings for a stock
base_url = 'https://www.sec.gov/Archives/'
ocf = ['Net cash provided by operating activities', 'Cash generated by operating activities']
title = 'CONSOLIDATED STATEMENTS OF CASH FLOWS'
oex = ['Purchases of property and equipment', 'Payments for acquisition of property, plant and equipment']
with open('./SEC/Master_Index/2020/QTR1.txt','r') as f:
    text = f.read()
dict_ind = json.loads(text)
cik = tickr_dict['META']['cik_str']
url = dict_ind[str(cik)+'_10-k']['File Name'].replace('-','').replace('.txt','')
for x in range(1,90):
    file = f'R{x}.htm'
    data = requests.get(base_url+url+f'/{file}',headers=SEC_headers).text
    if title in data:
        print(x)



In [ ]:
data = requests.get(base_url+url+f'/R8.htm',headers=SEC_headers).text
data

# Stock Analysis

In [ ]:
## Gather Data
def YfinanceData(Symbols,daymonthyear):
    # Yfinance_list = []
    Yfinance_dict = {}
    current_dir = os.getcwd()


    # Interval required 5 minutes
    start = dt.datetime(daymonthyear[2], daymonthyear[1], daymonthyear[0])

    for Symbol in Symbols:
        Yfinance_DataFrame = yf.download(tickers=Symbol, interval='1d', start= start)
        Yfinance_DataFrame['Ticker'] = Symbol
        # Yfinance_list.append(Yfinance_DataFrame)
        Yfinance_dict[Symbol] = Yfinance_DataFrame
        data_dir = os.path.join(current_dir,f'Stock_Data/{Symbol}_{daymonthyear[2]}{daymonthyear[1]}{daymonthyear[0]}.csv')
        Yfinance_DataFrame.to_csv(data_dir)
        
    return Yfinance_dict

In [ ]:
def StockPerformance(tickers,daymonthyear):
    current_dir = os.getcwd()
    analysis_dir = os.path.join(current_dir,'Stock_Analysis')
    data_dir = os.path.join(current_dir,'Stock_Data')
    for tckr in tickers:
        data = pd.read_csv(os.path.join(data_dir,f'{tckr}_{daymonthyear[2]}{daymonthyear[1]}{daymonthyear[0]}.csv'))
        
        ticker_dir = os.path.join(analysis_dir,tckr)
        try:
            os.mkdir(ticker_dir)
        except FileExistsError:
            pass
        
        fig = plt.figure()
        plt.plot(data['Close'])
        plt.title(f'{tckr}_{daymonthyear[2]}{daymonthyear[1]}{daymonthyear[0]}')
        plt.savefig(os.path.join(analysis_dir,tckr,f'{tckr}_{daymonthyear[2]}{daymonthyear[1]}{daymonthyear[0]}_plot.pdf'))

In [ ]:
# Specify stocks and Timeperiod
tickers = ['AMZN','GOOG','APPL']
daymonthyear = [30,12,2005]
data = YfinanceData(tickers,daymonthyear)
StockPerformance(tickers,daymonthyear)

In [ ]:
market_cap_data = web.get_quote_yahoo(tickers)['marketCap']

In [ ]:
tsla = yf.Ticker("TSLA")
tsla.info

In [ ]:
for x in range(3):
    print(x)

In [ ]:
import pandas as pd

df = pd.DataFrame({
    'Category': ['Fruit','Vegetable','Vegetable','Fruit','Vegetable','Vegetable','Fruit','Vegetable','Fruit','Vegetable'],
    'SubCategories': ['Apple','Brinjal','Brinjal','Apple','Carrot','Potato','Apple','Carrot','Banana','Brinjal'],
    'Count': [2,1,1,1,3,1,1,2,1,1],
})

df.set_index(['Category','SubCategories']).groupby(level=[0,1]).sum()

    

In [ ]:
dic ={"geeks": "A","for":"B","geeks":"C"}

value = {i for i in dic if dic[i]=="B"}
print("key by value:",value)


In [89]:
# Extracting relevances from tabular data

curr_dir = os.getcwd()

##### PARAMETERS ######
with open("SEC_config.yaml") as f:
    cfg = yaml.safe_load(f)

SEC_user_agent = cfg['SEC_user_agent']
encoding = cfg['encoding']
host = cfg['host']
SEC_headers = {'User-Agent': SEC_user_agent,'Accept-Encoding':'gzip','Host':host} #cfg['SEC_headers']
years = range(cfg['years']['start'],cfg['years']['end'])
params = cfg['params'] 
quarters = cfg['quarters']
stocks = ['AAPL']

file_types = ['_10-q','_10-k']
base_url = 'https://www.sec.gov/Archives/'
curr_dir = os.getcwd()
folder = 'SEC'

##############################
def cash_flow_statement_extraction(statement_quarter, df_statement):
    filt = df_statement['Category'] == 'Cash generated by operating activities'
    df_extract = df_statement.loc[filt]
    years = []
    months_ended = []
    quarters = []
    cfs = []

    for col in df_extract.columns:
        year, month_ended = extract_date_and_months(col)
        cf = df_extract[col].values[0]
        if (years == None) | (month_ended == None):
            continue

        years.append(year)
        months_ended.append(month_ended)
        quarters.append(statement_quarter)
        cfs.append(cf)

    cf_dict = {'cash_flow': cfs, 'year': years, 'quarter': quarters, 'months_ended': months_ended}
    return cf_dict

def extract_months(col_name):
    try:
        match = re.search(r'(\d+)\s+Months\s+Ended', col_name)
        return match.group(1) if match else None
    except:
        return None

def extract_date_and_months(col_name):
    year_match = re.search(r'\b\d{4}\b', col_name)
    months_match = re.search(r'\b\d+\sMonths\sEnded\b', col_name)
    
    year = year_match.group(0) if year_match else None
    months = months_match.group(0) if months_match else None
    
    extracted_months = extract_months(months)

    return year, extracted_months

if __name__ == "__main__":
    for stock in stocks:

        tab_data_path = os.path.join(curr_dir,'SEC',stock,'tabular_data')
        statements = os.listdir(tab_data_path) 
        df_stock_cf_10k = pd.DataFrame({'cash_flow': [], 'year': [], 'quarter': [], 'months_ended': []})
        df_stock_cf_10q = pd.DataFrame({'cash_flow': [], 'year': [], 'quarter': [], 'months_ended': []})

        for statement in statements:
            
            if statement.startswith('cash_flow_statement') & statement.endswith('_10-k.csv'):
                df_tab_data = pd.read_csv(os.path.join(tab_data_path,statement))
                cf_dict = cash_flow_statement_extraction(statement[-13:-9], df_tab_data)
                df_stock_cf_10k = pd.concat([df_stock_cf_10k,pd.DataFrame(cf_dict)])

            if statement.startswith('cash_flow_statement'):
                df_tab_data = pd.read_csv(os.path.join(tab_data_path,statement))
                cf_dict = cash_flow_statement_extraction(statement[-13:-9], df_tab_data)
                display(cf_dict)
                df_stock_cf_10q = pd.concat([df_stock_cf_10q,pd.DataFrame(cf_dict)])

        df_stock_cf_10k = df_stock_cf_10k.drop_duplicates().sort_values(by=['year'])
        df_stock_cf_10q = df_stock_cf_10q.drop_duplicates().sort_values(by=['year','quarter'])

{'cash_flow': [60098.0, 49481.0],
 'year': ['2020', '2019'],
 'quarter': ['QTR3', 'QTR3'],
 'months_ended': ['9', '9']}

{'cash_flow': [43827.0, 37845.0],
 'year': ['2020', '2019'],
 'quarter': ['QTR2', 'QTR2'],
 'months_ended': ['6', '6']}

{'cash_flow': [69391.0, 77434.0, 64225.0],
 'year': ['2019', '2018', '2017'],
 'quarter': ['QTR4', 'QTR4', 'QTR4'],
 'months_ended': ['12', '12', '12']}

{'cash_flow': [62744.0, 43827.0],
 'year': ['2021', '2020'],
 'quarter': ['QTR2', 'QTR2'],
 'months_ended': ['6', '6']}

{'cash_flow': [83838.0, 60098.0],
 'year': ['2021', '2020'],
 'quarter': ['QTR3', 'QTR3'],
 'months_ended': ['9', '9']}

...

{'cash_flow': [80674.0, 69391.0, 77434.0],
 'year': ['2020', '2019', '2018'],
 'quarter': ['QTR4', 'QTR4', 'QTR4'],
 'months_ended': ['12', '12', '12']}

{'cash_flow': [30516.0, 26690.0],
 'year': ['2019', '2018'],
 'quarter': ['QTR1', 'QTR1'],
 'months_ended': ['3', '3']}

{'cash_flow': [104038.0, 80674.0, 69391.0],
 'year': ['2021', '2020', '2019'],
 'quarter': ['QTR4', 'QTR4', 'QTR4'],
 'months_ended': ['12', '12', '12']}

{'cash_flow': [98024.0, 83838.0],
 'year': ['2022', '2021'],
 'quarter': ['QTR3', 'QTR3'],
 'months_ended': ['9', '9']}

{'cash_flow': [75132.0, 62744.0],
 'year': ['2022', '2021'],
 'quarter': ['QTR2', 'QTR2'],
 'months_ended': ['6', '6']}

In [80]:
def calculate_quarterly_cash_flow(df):
    df = df.sort_values(by=['year', 'quarter'])
    df['quarterly_cash_flow'] = 0.0
    
    for i in range(len(df)):
        if df.at[i, 'months_ended'] == 3:
            df.at[i, 'quarterly_cash_flow'] = df.at[i, 'cash_flow']
        elif df.at[i, 'months_ended'] == 6:
            if i >= 1 and df.at[i-1, 'months_ended'] == 3:
                df.at[i, 'quarterly_cash_flow'] = df.at[i, 'cash_flow'] - df.at[i-1, 'cash_flow']
        elif df.at[i, 'months_ended'] == 9:
            if i >= 2 and df.at[i-2, 'months_ended'] == 3 and df.at[i-1, 'months_ended'] == 6:
                df.at[i, 'quarterly_cash_flow'] = df.at[i, 'cash_flow'] - df.at[i-1, 'cash_flow'] - df.at[i-2, 'cash_flow']
        elif df.at[i, 'months_ended'] == 12:
            if i >= 3 and df.at[i-3, 'months_ended'] == 3 and df.at[i-2, 'months_ended'] == 6 and df.at[i-1, 'months_ended'] == 9:
                df.at[i, 'quarterly_cash_flow'] = df.at[i, 'cash_flow'] - df.at[i-1, 'cash_flow'] - df.at[i-2, 'cash_flow'] - df.at[i-3, 'cash_flow']
    
    return df

In [86]:
i = 2

In [87]:
df_stock_cf_10q.at[i, 'months_ended'] == 3

2    False
2    False
Name: months_ended, dtype: bool

In [81]:
filt = 
calculate_quarterly_cash_flow(df_stock_cf_10q)


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [88]:
df_stock_cf_10q

,cash_flow,year,quarter,months_ended
1,28293.0,2017,QTR1,3
2,64225.0,2017,QTR4,12
0,26690.0,2018,QTR1,3
1,43423.0,2018,QTR2,6
1,57911.0,2018,QTR3,9
1,77434.0,2018,QTR4,12
1,30516.0,2019,QTR1,3
1,37845.0,2019,QTR2,6
1,49481.0,2019,QTR3,9
0,69391.0,2019,QTR4,12


In [82]:
!pip list

Package             Version
------------------- -----------
appnope             0.1.4
asttokens           2.4.1
beautifulsoup4      4.12.3
certifi             2024.2.2
charset-normalizer  3.3.2
comm                0.2.2
contourpy           1.2.1
cycler              0.12.1
debugpy             1.8.1
decorator           5.1.1
exceptiongroup      1.2.1
executing           2.0.1
fonttools           4.52.4
idna                3.7
importlib_metadata  7.1.0
importlib_resources 6.4.0
ipykernel           6.29.4
ipython             8.18.1
jedi                0.19.1
jsons               1.6.3
jupyter_client      8.6.2
jupyter_core        5.7.2
kiwisolver          1.4.5
lxml                5.2.2
matplotlib          3.9.0
matplotlib-inline   0.1.7
nest-asyncio        1.6.0
numpy               1.26.4
packaging           24.0
pandas              2.2.2
pandas-datareader   0.10.0
parso               0.8.4
pexpect             4.9.0
pillow              10.3.0
pip                 21.2.3
platformdirs        